Create your first programming language that runs on .NET
=====

> Note: as of this writing, C# 11 is the latest version of C# to be released. C# 12 includes a feature that would make a lot of this code more concise - [collection expressions](https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-12.0/collection-expressions), however, I don't want to figure out how to get preview versions of .NET to work in a notebook. As such, I plan to update this when C# 12 releases.

Compilers can be a tricky topic to get into. There are [plenty](https://github.com/sandersn/mini-typescript) of [resources](https://archive.org/details/aho-compilers-principles-techniques-and-tools-2e_202203) available to learn compilers, but nothing I've found as small as a notebook.

This purpose of this notebook is to demonstrate an example of a tiny compiler.

Goals:
------
- Runs on .NET runtime as its own executable!
- variables, assignment, integer expression evaluation
- runs without the need of parser generators

Plan
----
This compiler follows 4 main phases of a canonical compiler:
1. Lexer
2. Parser
3. Semantic Analysis
4. Codegen

> Note: This language is extremely underwhelming. It supports no functions, only a single class of the most basic operators, has a single type of `int`, gives *zero* helpful error messages (it either works or it doesn't), and despite running on .NET, you can't explicitly import or call any of the .NET methods.

# Lexer
The first step in a compiler is a lexer. The idea is to take string text and convert it into a sequence of tokens, kind of like how you can take an English sentence and split it into words with `string.Split(" ")`.

We can't use `string.Split(" ")` for tokens, though, because sometimes tokens are not separated by spaces. For example, `x = 3` and `x=3` are both valid Python programs. Instead, we'll use something called regular expressions (`Regex`).

## Regex
A regex is a pattern with which we can test whether another string conforms to that pattern. For example, the string `name` matches the pattern `name`. In a regex expression, the unescaped `.` means "match any single character". So therefore, `name` also matches the pattern `n.me`, but so so the strings `nome` and `neme`.

In [24]:
using System.Text.RegularExpressions;

string[] words = { "name", "neme", "nome", "neem" };
string pattern = "n.me";

words.Select(w => new { String = w, IsMatch = Regex.IsMatch(w, pattern) }).DisplayTable()

String,IsMatch
name,True
neme,True
nome,True
neem,False



Another reason we might want to use regex to break strings into tokens ("sentence into words") is because it helps us classify our tokens into *types* or *categories*. In English sentences, words have are classified into "parts of speech", like "verb" and "noun". In a naive example where words fall into one part of speech, we can look up words in a sentence in a dictionary to find their parts of speech. For example, in `I ran to the store`, `I` is a noun, `ran` is a past-tense verb, etc. With regex, we actually have it a bit easier, as we can classify tokens based on their *patterns* instead of looking them up in a dictionary. For example, the pattern `-?[0-9]+` matches integers like `1` and `626`, and the pattern `^[a-zA-Z]+[0-9a-zA-Z]*` matches IDs like `index` and `age`. 

First, we'll define our token types using an `enum`, and a `record struct` to store our token type, along with an `int Start` and `ReadOnlyMemory<char> Memory` to reference where it lives in memory.

> (we could use this information in the future for error reporting, though we do not in this example) 

In [25]:
public enum TokenType { Semicolon, Assignment, LeftParenthesis, RightParenthesis, Add, Subtract, Multiply, Divide, Id, Integer, Whitespace }
public record struct Token(int Start, ReadOnlyMemory<char> Memory, TokenType Type);

We also want to associate token types with their corresponding `Regex` rules.

We could use a `Dictionary` (hashmap) here, but as we'll see, we'll be iterating through our collection of rules, so it makes more sense to just defined them as a list of tuples. 

In [26]:
(TokenType Type, Regex Pattern)[] rules = new (TokenType type, Regex regex)[] {
    (TokenType.Semicolon,           new(@"^;", RegexOptions.Compiled)),
    (TokenType.Assignment,          new(@"^=", RegexOptions.Compiled)),
    (TokenType.LeftParenthesis,     new(@"^\(", RegexOptions.Compiled)),
    (TokenType.RightParenthesis,    new(@"^\)", RegexOptions.Compiled)),
    (TokenType.Add,                 new(@"^\+", RegexOptions.Compiled)),
    (TokenType.Subtract,            new(@"^\-", RegexOptions.Compiled)),
    (TokenType.Multiply,            new(@"^\*", RegexOptions.Compiled)),
    (TokenType.Divide,              new(@"^/", RegexOptions.Compiled)),
    (TokenType.Id,                  new(@"^[a-zA-Z]+[0-9a-zA-Z]*", RegexOptions.Compiled)),
    (TokenType.Integer,             new(@"^-?[0-9]+", RegexOptions.Compiled)),
    (TokenType.Whitespace,          new(@"^[ \n\t\r]", RegexOptions.Compiled))
};

In our case, we can modify our use of `Regex` to "pull tokens off" when we have at least one match.



With this knowledge, we can imagine a simple algorithm to break off tokens:
```
tokens = []

while input string s not empty
    for each (type, regex pattern) in rules
        search for first match m in regex pattern
        if m exists
            add (m, type) to tokens
        if m not exists
            error

return tokens
```

which is exactly what we do below

First we define our input string:

In [27]:
public ReadOnlyMemory<char> input =
    """
    a = 2;
    b = 3;
    c = (3 + a) * b;

    c
    """.AsMemory();

You'll note that instead of using `input`, we call `AsMemory` and store it in `feed`.

`feed`, like `Token.Memory`, is a `ReadOnlySpan<Memory>`. It allows us to create a wrapper over our original input string and slice the front off each time we pull off a token without copying any of the input string.

You could write a similar implementation with only `input` and keeping track of an index as you go along, but passing subslices of `string`s and `Array`s to other functions becomes tedious and error prone as complexity increases, so I prefer using `Span<T>` and `Memory<T>` types.

In [28]:
#nullable enable
public List<Token>? Lex(ReadOnlyMemory<char> feed) 
{
    List<Token> tokens = new();

    while (feed.Length > 0)
    {
        // get the first match
        foreach (var rule in rules)
        {
            var match = rule.Pattern.EnumerateMatches(feed.Span);
            if (match.MoveNext())
            {
                tokens.Add(new Token(input.Length - feed.Length, feed[0..match.Current.Length], rule.Type));
                feed = feed[match.Current.Length..];
                goto next;
            }
        }
        
        Console.WriteLine("Error at: " + ((Func<Token, string>)(t => $"{t.Start} {t.Type} {t.Memory}"))(tokens[^1]));
        return null;
        next:;
    }

    // remove whitespace
    return tokens.Where(t => t.Type != TokenType.Whitespace).ToList();
}

var tokens = Lex(input);
tokens

index value 0 Token { Start = 0, Memory = a, Type = Id } Start 0 Memory a Type Id 1 Token { Start = 2, Memory = =, Type = Assignment } Start 2 Memory = Type Assignment 2 Token { Start = 4, Memory = 2, Type = Integer } Start 4 Memory 2 Type Integer 3 Token { Start = 5, Memory = ;, Type = Semicolon } Start 5 Memory ; Type Semicolon 4 Token { Start = 8, Memory = b, Type = Id } Start 8 Memory b Type Id 5 Token { Start = 10, Memory = =, Type = Assignment } Start 10 Memory = Type Assignment 6 Token { Start = 12, Memory = 3, Type = Integer } Start 12 Memory 3 Type Integer 7 Token { Start = 13, Memory = ;, Type = Semicolon } Start 13 Memory ; Type Semicolon 8 Token { Start = 16, Memory = c, Type = Id } Start 16 Memory c Type Id 9 Token { Start = 18, Memory = =, Type = Assignment } Start 18 Memory = Type Assignment 10 Token { Start = 20, Memory = (, Type = LeftParenthesis } Start 20 Memory ( Type LeftParenthesis 11 Token { Start = 21, Memory = 3, Type = Integer } Start 21 Memory 3 Type Integer 12 Token { Start = 23, Memory = +, Type = Add } Start 23 Memory + Type Add 13 Token { Start = 25, Memory = a, Type = Id } Start 25 Memory a Type Id 14 Token { Start = 26, Memory = ), Type = RightParenthesis } Start 26 Memory ) Type RightParenthesis 15 Token { Start = 28, Memory = *, Type = Multiply } Start 28 Memory * Type Multiply 16 Token { Start = 30, Memory = b, Type = Id } Start 30 Memory b Type Id 17 Token { Start = 31, Memory = ;, Type = Semicolon } Start 31 Memory ; Type Semicolon 18 Token { Start = 36, Memory = c, Type = Id } Start 36 Memory c Type Id

Great! Our input was an input `string` and now we have a `List<Token> tokens`!

# Parser
Once we have our string of tokens, the next step is the parser.

The input of the parser is our string of tokens, and the output is something called an **Abstract Syntax Tree (AST)**. The AST is formed by walking through the grammar (at least on a recursive descent parser), and if you've done some reading on this topic, there is a possible point of confusion here - but I'll get back to that in a bit.


## Grammar
I used the term "grammar" earlier without defining it. A *grammar* simply defines the rules of a language. Every language has a grammar, for example, take a possible subset of English grammar:
```
Sentence    -> Subject Verb Object
Subject     -> i | he | they
Verb        -> run | drive
Object      -> Prepositional Phrase | Noun
Prepositional Phrase -> to the Noun
Noun        -> store | post office
```

The capital letters are called *non-terminals*, which means that they have one or more *production rules*, and can be *expanded* into the right hand side of one of their production rules. The lowercase letters are called *terminals* which is basically just the same thing as tokens honestly in that they cannot be expanded with production rules and therefore make up the final sentence of the grammar.  

We can create an example sentence can be produced by running through rules of the grammar. For example, for the sentence `i run to the store`,

```
Sentence
Subject Verb Object
i Verb Object
i run Object
i run Prepositional Phrase
i run to the Noun
i run to the store
```

You may note two relevant points here:
1. I followed an algorithm which is "**always expand the left-most non-terminal first with the left-most rule**"
2. You could expand the production of this sentence into a tree


In [29]:
graph TD
    A[Sentence] --> B[Subject]
    A --> C[Verb]
    A --> D[Object]
    B --> E[i]
    C --> F[run]
    D --> G[Prepositional Phrase]
    G --> H[to the]
    G --> I[Noun]
    I --> J[store]

For the first point, a modified version of this algorithm is how we're going to actually parse our sentence. In the above example, `i run to the store` happened to be the sentence we get when we follow our algorithm. But imagine we weren't trying to randomly produce sentences, but instead get a specific sentence at the end *to make sure it conforms to the rules of the grammar* (are your clocks ticking?). There are two modifications we need to make, and one important point to keep in mind:
1. Keep a pointer to our list of tokens as we go along.
2. When we reach a terminal (at a leaf), compare it against the terminal pointed to by our pointer. If they are the same, continue with the algorithm. If not, rewind, remembering to move our pointer back as needed.

### Left Recursion
> Important Point: __no non-terminal can have a rule where its own non-terminal is the first symbol in the grammar__
>
> Why? Our algorithm will keep looping forever. For example:
> 
> grammar:
> ```
> S -> E
> E -> E + E
> ```
> 
> derivation:
> ```
> S
> E
> E + E
> E + E + E
> E + E + E + E
> ...
> ```


Here is the first part of our grammar:
```
Program                 -> Statements | Expression
Statements              -> Statement Statements
Statement               -> Assignment Statement
Assignment Statement    -> id = Expression;
```

### Ambiguity
Another important point to note is that most languages that exist have some form of *ambiguity*, which means that for at least one sentence in the language, there is more than one way to derive rules to produce that sentence. Take the following example:

grammar:
```
E   ->
    E + E
    E * E
    E
```

target sentence:
```
1 + 2 * 3
```

(Parentheses added for visual clarification)<br>derivation 1:
```
E
(E + E)
(E + (E * E))
1 + 2 * 3
```

derivation 2:
```
E
(E * E)
((E + E) * E)
1 + 2 * 3
```

You'll also note we broke our "no left-recursion" rule. Sometimes ambiguity is okay, but in this instance, you'll note that it actually changes the meaning of the sentence (`(1 + (2 * 3))` = `7`, `((1 + 2) * 3)` = `9`). I point this out because often times, when we have ambiguity and left-recursion, sometimes we can knock both problems out at the same time by introducing new non-terminals:

```
Expression   ->
    Term + Expression
    Term - Expression
    Term
Term   ->
    Factor * Term
    Factor / Term
    Factor
Factor  ->
    (Expression) | id | int
```

This is actually the second part of our final grammar.

> Note: I think it's important to note here that I am skipping over entire classes of parsers and grammars.
> 
> The grammar I am describing here is called "**LL**", which means it reads tokens from left-to-right and constructs a "left-most derivation" (just tries doing all the rules and stuff from the left first), and the parser I am writing is called "manual recursive descent". It is also "top-down", meaning it starts from the first non-terminal and tries to produce a matching sentence.
> 
> A point of inefficiency for the parser I'm writing is that it tries all the rules until it finds a match, meaning that if it does *not* find a match, it tries every single navigation of productions before giving up. This isn't *terrible*, as production compilers often use a mixture of parsing techniques with recursive descent often being the primary one because of the flexibility it provides, but it is worth mentioning that there are other parsing techniques that eliminate some of these inefficiencies by requiring you to be stricter with how you define your grammar.


And about that "point of confusion" I mentioned earlier:

The path taken to navigate the grammar to result in the final sentence is often called a *parse tree*. This is an actual tree you can draw from the rules of the grammar, and is often very helpful for when you're confused (see Mermaid tree above).

However, it is *different* from the **AST**, in that your AST is a data structure you use in the next phase of your parser and your syntax tree is (probably) not. It doesn't mean that you can't create a data structure for a syntax tree or that doing so would never be helpful, but it's often easier to just produce your AST as you walk along your grammar.

Now back to code.

In [30]:
using System.Collections.Immutable;

public record Program(ImmutableList<Statement> Statements, Expression Eval);
public record Statement();
public record AssignmentStatement(Token Identifier, Expression Expression) : Statement();
public record Expression();
public record NameExpression(Token Identifier) : Expression();
public record BinaryExpression(Expression Left, Token Operator, Expression Right) : Expression();
public record IntLiteralExpression(int Value) : Expression();

We can define our types using `record`s in C#. `record`s are kind of like classes, but they can be defined in a single statement. The member definitions passed in the `()`s become `readonly` auto-properties, meaning they are immutable.

In a manual recursive descent parser, you can define your grammar rules directly as code.

This code tries to parse the next token as an integer literal, returning the `IntLiteralExpression` node of our tree if it succeeds and `null` if it fails, also writing to `rest` to move our pointer to the next token appropriately.

Like our general algorithm above with pointers, we need to move our pointer up on success and move it back on fail, so the the weird `!= null` comparisons are to ensure we do that while keeping everything an expression. This code would probably look a lot cleaner in F#.

In [31]:
#nullable enable
static Expression? ParseLiteral(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ((rest = tokens) != null
    && tokens is [{ Type: TokenType.Integer } token, ..var _rest]
    && int.TryParse(token.Memory.Span, out int value)
    && (rest = _rest) != null)
    ?
    new IntLiteralExpression(value)
    :
    null;

ParseLiteral(Lex("3".AsMemory()).ToArray().AsSpan(), out _)

Value,3


We can continue to define more rules so we can parse expressions:

In [32]:
#nullable enable
static Expression? ParseExpression(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ParseTerm(tokens, out rest) switch
    {
        { } left => rest switch
            {
                [{ Type: TokenType.Add or TokenType.Subtract } op, ..var _rest]
                    when ParseTerm(_rest, out rest) is { } right => new BinaryExpression(left, op, right),
                _ => left
            },
        _ => null
    };

static Expression? ParseTerm(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ParseFactor(tokens, out rest) switch
    {
        { } left => rest switch
            {
                [{ Type: TokenType.Multiply or TokenType.Divide } op, ..var _rest]
                    when ParseFactor(_rest, out rest) is { } right => new BinaryExpression(left, op, right),
                _ => left
            },
        _ => null
    };

static Expression? ParseFactor(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ParseWrappedExpression(tokens, out rest)
    ?? ParseLiteral(tokens, out rest)
    ?? (tokens is [{ Type: TokenType.Id }, ..var _rest] && (rest = _rest) != null
        ? new NameExpression(tokens[0])
        : null);

static Expression? ParseWrappedExpression(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ((rest = tokens) != null
    && tokens is [{ Type: TokenType.LeftParenthesis } left, ..var _rest]
    && ParseExpression(_rest, out var __rest) is { } expression
    && __rest is [{ Type: TokenType.RightParenthesis } right, ..var ___rest]
    && (rest = ___rest) != null)
    ?
    expression
    :
    null;

ParseExpression(Lex("(3 + 4)".AsMemory()).ToArray().AsSpan(), out _)

BinaryExpression { Left = IntLiteralExpression { Value = 3 }, Operator = Token { Start = 33, Memory = +, Type = Add }, Right = IntLiteralExpression { Value = 4 } } Left IntLiteralExpression { Value = 3 } Value 3 Operator Token { Start = 33, Memory = +, Type = Add } Start 33 Memory + Type Add Right IntLiteralExpression { Value = 4 } Value 4

And statements:

In [33]:
#nullable enable
static ImmutableList<Statement> ParseStatements(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ParseStatement(tokens, out rest) switch
        {
            null => ImmutableList<Statement>.Empty,
            var statement => ParseStatements(rest, out rest).Insert(0, statement)
        };

static Statement? ParseStatement(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => ParseAssignment(tokens, out rest);

static AssignmentStatement? ParseAssignment(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    =>  (rest = tokens).Length > 0
    &&  tokens is [{ Type: TokenType.Id }, { Type: TokenType.Assignment }, ..var _rest]
    &&  ParseExpression(_rest, out var __rest) is { } expression
    &&  __rest is [{ Type: TokenType.Semicolon }, ..var ___rest] && (rest = ___rest) != null // how do i make this check a no-op
        ? new AssignmentStatement(tokens[0], expression)
        : null;

ParseStatements(Lex("a = 3;".AsMemory()).ToArray().AsSpan(), out _)

index value 0 AssignmentStatement { Identifier = Token { Start = 31, Memory = a, Type = Id }, Expression = IntLiteralExpression { Value = 3 } } Identifier Token { Start = 31, Memory = a, Type = Id } Start 31 Memory a Type Id Expression IntLiteralExpression { Value = 3 } Value 3

**bug is that we don't reject properly when invalid program**

And finally our entire program:

In [34]:
#nullable enable
static Program? Parse(ReadOnlySpan<Token> tokens, out ReadOnlySpan<Token> rest)
    => (ParseStatements(tokens, out rest) is ImmutableList<Statement> statements
    && ParseExpression(rest, out rest) is var expression)
    ? new Program(statements, expression)
    : null;

Program? program = Parse(tokens.ToArray().AsSpan(), out _);

program

Program { Statements = System.Collections.Immutable.ImmutableList`1[Submission#28+Statement], Eval = NameExpression { Identifier = Token { Start = 36, Memory = c, Type = Id } } } Statements index value 0 AssignmentStatement { Identifier = Token { Start = 0, Memory = a, Type = Id }, Expression = IntLiteralExpression { Value = 2 } } Identifier Token { Start = 0, Memory = a, Type = Id } Start 0 Memory a Type Id Expression IntLiteralExpression { Value = 2 } Value 2 1 AssignmentStatement { Identifier = Token { Start = 8, Memory = b, Type = Id }, Expression = IntLiteralExpression { Value = 3 } } Identifier Token { Start = 8, Memory = b, Type = Id } Start 8 Memory b Type Id Expression IntLiteralExpression { Value = 3 } Value 3 2 AssignmentStatement { Identifier = Token { Start = 16, Memory = c, Type = Id }, Expression = BinaryExpression { Left = BinaryExpression { Left = IntLiteralExpression { Value = 3 }, Operator = Token { Start = 23, Memory = +, Type = Add }, Right = NameExpression { Identifier = Token { Start = 25, Memo... Identifier Token { Start = 16, Memory = c, Type = Id } Start 16 Memory c Type Id Expression BinaryExpression { Left = BinaryExpression { Left = IntLiteralExpression { Value = 3 }, Operator = Token { Start = 23, Memory = +, Type = Add }, Right = NameExpression { Identifier = Token { Start = 25, Memory = a, Type = Id } } }, Operator = Token { Start = 28, Memory = *, Type = Multiply }, Right ... Left BinaryExpression { Left = IntLiteralExpression { Value = 3 }, Operator = Token { Start = 23, Memory = +, Type = Add }, Right = NameExpression { Identifier = Token { Start = 25, Memory = a, Type = Id } } } Left IntLiteralExpression { Value = 3 } Value 3 Operator Token { Start = 23, Memory = +, Type = Add } Start 23 Memory + Type Add Right NameExpression { Identifier = Token { Start = 25, Memory = a, Type = Id } } Identifier Token { Start = 25, Memory = a, Type = Id } Operator Token { Start = 28, Memory = *, Type = Multiply } Start 28 Memory * Type Multiply Right NameExpression { Identifier = Token { Start = 30, Memory = b, Type = Id } } Identifier Token { Start = 30, Memory = b, Type = Id } Start 30 Memory b Type Id Eval NameExpression { Identifier = Token { Start = 36, Memory = c, Type = Id } } Identifier Token { Start = 36, Memory = c, Type = Id } Start 36 Memory c Type Id

# Semantic Analysis
Now that we have our AST, the next phase of the compiler is semantic analysis.

The role of semantic analysis can change depending on how you define your language. Your language, like F#, might provide rich type inference and strict type checking, and here is where you'll do most of that checking. You can also do some high-level optimization during this phase, like [constant folding](https://en.wikipedia.org/wiki/Constant_folding), for example.

For our purposes, we're just going to do the important yet humble act of making sure we don't use a variable before it is defined and don't define one multiple times.

In [35]:
#nullable enable
static List<string>? GetSymbols(Program program)
{
    var symbols = new List<string>();

    foreach (Statement statement in program.Statements)
    {
        if (statement is AssignmentStatement assignment)
        {
            // multiply defined
            if (symbols.Contains(assignment.Identifier.Memory.ToString()))
                return null;
            
            if (IsValidExpression(symbols, assignment.Expression) is false)
                return null;
            
            // check expression
            symbols.Add(assignment.Identifier.Memory.ToString());
        }
    }

    return symbols;

    static bool IsValidExpression(List<string> symbols, Expression expression)
        => expression switch
        {
            NameExpression name => symbols.Contains(name.Identifier.Memory.ToString()),
            BinaryExpression binary => IsValidExpression(symbols, binary.Left) 
                && IsValidExpression(symbols, binary.Right),
            IntLiteralExpression _ => true,
            _ => false
        };
}

var symbols = GetSymbols(program);

symbols

[ a, b, c ]

# Codegen
For codegen, we're going to be using [AsmResolver](https://github.com/Washi1337/AsmResolver) to emit [CIL](https://en.wikipedia.org/wiki/Common_Intermediate_Language).

In [36]:
#r "nuget: AsmResolver.DotNet, 5.4.0"

using AsmResolver.DotNet;
using AsmResolver.DotNet.Code.Cil;
using AsmResolver.DotNet.Signatures;
using AsmResolver.DotNet.Signatures.Types;
using AsmResolver.PE.DotNet.Cil;
using AsmResolver.PE.DotNet.Metadata.Tables.Rows;

Installed Packages AsmResolver.DotNet, 5.4.0

We're going to need to write some boilerplate to define an assembly and module, and define our entry point.

In [37]:
var assembly = new AssemblyDefinition(
    name: "MyAssembly",
    version: new(1, 0, 0, 0));

var runtime = DotNetRuntimeInfo.Parse(".NETCoreApp,Version=v7.0");
var module = new ModuleDefinition("MyAssembly.exe", runtime.GetDefaultCorLib());
var factory = module.CorLibTypeFactory;

assembly.Modules.Add(module);

var programType = new TypeDefinition(
    ns: "", 
    name: "Program", 
    TypeAttributes.Class,
    baseType: factory.Object.Type);

module.TopLevelTypes.Add(programType);

var mainMethod = new MethodDefinition(
    name: "Main",
    MethodAttributes.Public | MethodAttributes.Static,
    signature: MethodSignature.CreateStatic(returnType: module.CorLibTypeFactory.Void));

var methodBody = new CilMethodBody(mainMethod);
mainMethod.CilMethodBody = methodBody;

var write = 
    module.DefaultImporter.ImportType(typeof(Console))
    .CreateMemberReference("Write", MethodSignature.CreateStatic(
        factory.Void, factory.Int32))
    .ImportWith(module.DefaultImporter);

## Symbol Table
CIL uses a [stack evaluation model](https://en.wikipedia.org/wiki/Stack_machine), meaning that we have a stack to store values on and an "unlimited" (256) number of local variables. The .NET runtime then converts this IL to assembly code at runtime, though it is possible to compile it to native code ahead of time (AOT).

Inside our method body, we need to initialize our symbols as stack local variables, and then setup a `Dictionary<string, int>` to reference them by when generating code.

What I am creating with this `Dictionary` is something called a **symbol table**, but a very simplified watered down version. In real programming languages, you can have any number of scopes, and variables can be defined with the same name as another as long as they are uniquely defined in their own scope. A data structure for this model looks a lot more like a `Dictionary` where each value is a list.

> Note: In most languges, module-like scopes can be named, meaning that identifiers inside modules must be *qualified* with the name of their containing scope. Some languages, like F#, Java and TypeScript, have a notion of "opening" a module which means adding all the items inside it to the lexical environment it's opened in.

In [38]:
var symbolTuples = symbols.Select((s, i) => (Var: new CilLocalVariable(factory.Int32), Id: s, Index: i));

foreach (var tpl in symbolTuples)
{
    methodBody.LocalVariables.Add(tpl.Var);
}

var symbolTable = symbolTuples.ToDictionary(k => k.Id, v => v.Index);

Now we're ready to start generating code.

> You might recall that our language always ends with an expression. This type of pattern is common when using languages in a scripting environment, such as Python in an interactive environment and [C# Script](https://learn.microsoft.com/en-us/archive/msdn-magazine/2016/january/essential-net-csharp-scripting). For our purposes, I didn't want to go through the work / add the complexity of implementing calling .NET methods, so in every program, `Console.Write` is called on this final expression. This gives us an easy way to test our programs, but is obviously lacking, as we cannot call `Console.Write` in various points of our code. 

We have `n` statements and one expression at the end, and it would make sense that - since we don't have functions - we generate code for our language in order.

So our general algorithm would look something like
```
for each statement s
    codegen(s)
codegen(final expression)
codegen(call `Console.Write`)
```

The way a stack evaluation model works, is that the "current" value is always at the top of the stack. Looking at the last two instructions above, we see we never "pass" the final expression to `Console.Write`, but instead push the argument onto the stack and then call `Console.Write`. This is also how we can handle assignment statements - by pushing the expression onto the stack and then popping it into the local variable being assigned to.

First, we define a method for evaluating expressions and pushing them onto the stack: 

In [39]:
List<CilInstruction> Codegen(Expression expression)
    => expression switch
    {
        NameExpression s => new List<CilInstruction>()
        {
            new CilInstruction(CilOpCodes.Ldloc_S, methodBody.LocalVariables[symbolTable[s.Identifier.Memory.ToString()]]),
        },
        BinaryExpression s => ((Func<List<CilInstruction>>)(() =>
        {    
            var l = new List<CilInstruction>();
            l.AddRange(Codegen(s.Left));
            l.AddRange(Codegen(s.Right));
            l.Add(new CilInstruction(s.Operator.Type switch
            {
                TokenType.Add => CilOpCodes.Add,
                TokenType.Subtract => CilOpCodes.Sub,
                TokenType.Multiply => CilOpCodes.Mul,
                TokenType.Divide => CilOpCodes.Div,
                _ => throw new NotImplementedException()
            }));

            return l;
        }))(),
        IntLiteralExpression s => new List<CilInstruction>()
        {
            new CilInstruction(CilOpCodes.Ldc_I4, s.Value),
        },
        _ => throw new NotImplementedException()
    };

Then to generate code for each statement, where every statement is an assignment statement, we generate code to push the expression onto the stack, then code to pop the stack into a local variable.

In [40]:
var il = methodBody.Instructions;

il.AddRange(
    program.Statements
        .SelectMany(s => s switch
            {
                AssignmentStatement { Identifier: { Memory: var memory }, Expression: Expression expression } => ((Func<List<CilInstruction>>)(() =>
                {    
                    var l = new List<CilInstruction>();
                    l.AddRange(Codegen(expression));
                    l.Add(new CilInstruction(CilOpCodes.Stloc_S, methodBody.LocalVariables[symbolTable[memory.ToString()]]));

                    return l;
                }))(),
                _ => throw new NotImplementedException()
            }    
        )
);

And finally, the evaluate the last expression and call `Console.Write`:

In [41]:
il.AddRange(Codegen(program.Eval));
il.Add(CilOpCodes.Call, write);
il.Add(new CilInstruction(CilOpCodes.Ret));

Now we can set our `mainMethod` as our entry point

In [42]:
programType.Methods.Add(mainMethod);
module.ManagedEntryPointMethod = mainMethod;

And write our assembly (it is a framework-dependent assembly, so we also need to write a `*.runtimeconfig.json`)

In [43]:
assembly.Write("MyAssembly.exe");
System.IO.File.WriteAllText("MyAssembly.runtimeconfig.json",
"""
{
   "runtimeOptions": {
     "tfm": "net7.0",
     "framework": {
       "name": "Microsoft.NETCore.App",
       "version": "7.0.0"
     }
   }
}
"""
);

Now, if we run our assembly with PowerShell:

In [44]:
dotnet MyAssembly.exe

15


For seeing our compiled code, I prefer using [ILSpy](https://github.com/icsharpcode/ILSpy) (can install on Windows with `winget install ILSpy`), but if you don't want to install that tool, we can pretty print our instructions with AsmResolver like so (taken from [here](https://docs.washi.dev/asmresolver/guides/dotnet/managed-method-bodies.html#pretty-printing-cil-instructions)):

In [45]:
var formatter = new CilInstructionFormatter();
foreach (CilInstruction instruction in methodBody.Instructions)
    Console.WriteLine(formatter.FormatInstruction(instruction));

IL_0000: ldc.i4 2
IL_0005: stloc.s V_0
IL_0007: ldc.i4 3
IL_000C: stloc.s V_1
IL_000E: ldc.i4 3
IL_0013: ldloc.s V_0
IL_0015: add
IL_0016: ldloc.s V_1
IL_0018: mul
IL_0019: stloc.s V_2
IL_001B: ldloc.s V_2
IL_001D: call System.Void System.Console::Write(System.Int32)
IL_0022: ret
